In [1]:
## 기본 library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
## model library
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from keras import models, layers, optimizers
## helper library
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from bayes_opt import BayesianOptimization
import warnings
warnings.filterwarnings(action='ignore')

from sklearn.datasets import load_breast_cancer
from sklearn.cluster import KMeans

d = load_breast_cancer()

data = pd.DataFrame(d.data, columns=d.feature_names)
target = pd.DataFrame(d.target)



Using TensorFlow backend.


In [2]:
import tensorflow as tf
tf.compat.v1.reset_default_graph()
tf.random.set_seed(0)
np.random.seed(0)

In [3]:
std = StandardScaler()
for c in data.columns:
    data[c] = std.fit_transform(data[c].values.reshape(-1, 1))
    
train_X, test_X, train_y, test_y = train_test_split(data, target, test_size=0.3, random_state=0, shuffle=True)

In [4]:
train_X = train_X.reset_index().drop('index', axis=1)
train_y = train_y.reset_index().drop('index', axis=1)
test_X = test_X.reset_index().drop('index', axis=1)
test_y = test_y.reset_index().drop('index', axis=1)

In [5]:
# lr = LogisticRegression(penalty='l2', random_state=0).fit(train_X, train_y)
# svc = SVC(kernel='rbf', random_state=0).fit(train_X, train_y)
# tree = DecisionTreeClassifier(random_state=0).fit(train_X, train_y)
# knn = KNeighborsClassifier(n_neighbors=10).fit(train_X, train_y)

# tr_lr = pd.DataFrame(lr.predict(train_X), columns=['lr'])
# te_lr = pd.DataFrame(lr.predict(test_X), columns=['lr'])

# tr_svc = pd.DataFrame(svc.predict(train_X), columns=['svc'])
# te_svc = pd.DataFrame(svc.predict(test_X), columns=['svc'])

# tr_tree = pd.DataFrame(tree.predict(train_X), columns=['tree'])
# te_tree = pd.DataFrame(tree.predict(test_X), columns=['tree'])

# tr_knn = pd.DataFrame(knn.predict(train_X), columns=['knn'])
# te_knn = pd.DataFrame(knn.predict(test_X), columns=['knn'])

# tr_pr = pd.concat([tr_lr, tr_svc, tr_tree, tr_knn], axis=1)
# te_pr = pd.concat([te_lr, te_svc, te_tree, te_knn], axis=1)

In [6]:
lr = LogisticRegression(penalty='l2', random_state=0).fit(train_X, train_y)
svc = SVC(kernel='rbf', random_state=0).fit(train_X, train_y)
tree = DecisionTreeClassifier(random_state=0, max_depth=9).fit(train_X, train_y)
knn = KNeighborsClassifier(n_neighbors=10).fit(train_X, train_y)

tr_lr = pd.DataFrame(list(list(zip(*lr.predict_proba(train_X)))[0]), columns=['lr'])
te_lr = pd.DataFrame(list(list(zip(*lr.predict_proba(test_X)))[0]), columns=['lr'])

tr_tree = pd.DataFrame(list(list(zip(*tree.predict_proba(train_X)))[0]), columns=['tree'])
te_tree = pd.DataFrame(list(list(zip(*tree.predict_proba(test_X)))[0]), columns=['tree'])

tr_knn = pd.DataFrame(list(list(zip(*knn.predict_proba(train_X)))[0]), columns=['knn'])
te_knn = pd.DataFrame(list(list(zip(*knn.predict_proba(test_X)))[0]), columns=['knn'])

tr_pr = pd.concat([tr_lr, tr_tree, tr_knn], axis=1)
te_pr = pd.concat([te_lr,te_tree, te_knn], axis=1)

In [7]:
tree.feature_importances_

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.02112487, 0.        , 0.01815419, 0.        ,
       0.        , 0.        , 0.        , 0.04749283, 0.01058659,
       0.        , 0.        , 0.        , 0.        , 0.01016418,
       0.        , 0.02784245, 0.        , 0.08615112, 0.00990228,
       0.        , 0.01032055, 0.728114  , 0.03014695, 0.        ])

In [8]:
tf.compat.v1.reset_default_graph()
K.clear_session()

i1 = layers.Input(shape=(train_X.shape[1:]))
i2 = layers.Input(shape= (tr_pr.shape[1:]))

x = layers.Dense(64, activation='relu', activity_regularizer='l2')(i1)
# x2 = layers.Dense(32, activation='relu', activity_regularizer='l2')(i2)

x = layers.Dense(16, activation='relu')(x)

x = layers.concatenate([x, i2])

out = layers.Dense(1, activation='sigmoid')(x)

model = models.Model([i1, i2], out)

model.compile(optimizer = 'adam',
             loss ='binary_crossentropy',
             metrics=['acc'])

In [9]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 64)           1984        input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 16)           1040        dense_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 3)            0                                            
____________________________________________________________________________________________

In [10]:
es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [11]:
model.fit([train_X, tr_pr], train_y, 
         epochs=300,
         validation_split=0.3,
         callbacks=[es]
         )

Train on 278 samples, validate on 120 samples
Epoch 1/300
278/278 [==============================] - 0s 2ms/step - loss: 5.7402 - acc: 0.6583 - val_loss: 4.9569 - val_acc: 0.8250
Epoch 2/300
278/278 [==============================] - 0s 111us/step - loss: 4.5949 - acc: 0.8453 - val_loss: 3.9530 - val_acc: 0.8833
Epoch 3/300
278/278 [==============================] - 0s 108us/step - loss: 3.6195 - acc: 0.9065 - val_loss: 3.2358 - val_acc: 0.9250
Epoch 4/300
278/278 [==============================] - 0s 108us/step - loss: 3.0351 - acc: 0.9388 - val_loss: 2.7159 - val_acc: 0.9667
Epoch 5/300
278/278 [==============================] - 0s 108us/step - loss: 2.5297 - acc: 0.9496 - val_loss: 2.3238 - val_acc: 0.9667
Epoch 6/300
278/278 [==============================] - 0s 120us/step - loss: 2.2058 - acc: 0.9532 - val_loss: 2.0325 - val_acc: 0.9667
Epoch 7/300
278/278 [==============================] - 0s 111us/step - loss: 1.8779 - acc: 0.9532 - val_loss: 1.7988 - val_acc: 0.9667
Epoch 8/300

Epoch 61/300
278/278 [==============================] - 0s 140us/step - loss: 0.1498 - acc: 0.9964 - val_loss: 0.2144 - val_acc: 0.9833
Epoch 62/300
278/278 [==============================] - 0s 126us/step - loss: 0.1473 - acc: 0.9964 - val_loss: 0.2121 - val_acc: 0.9667
Epoch 63/300
278/278 [==============================] - 0s 126us/step - loss: 0.1437 - acc: 0.9964 - val_loss: 0.2090 - val_acc: 0.9667
Epoch 64/300
278/278 [==============================] - 0s 111us/step - loss: 0.1403 - acc: 0.9964 - val_loss: 0.2044 - val_acc: 0.9833
Epoch 65/300
278/278 [==============================] - 0s 122us/step - loss: 0.1375 - acc: 0.9964 - val_loss: 0.2011 - val_acc: 0.9833
Epoch 66/300
278/278 [==============================] - 0s 126us/step - loss: 0.1344 - acc: 0.9964 - val_loss: 0.2001 - val_acc: 0.9750
Epoch 67/300
278/278 [==============================] - 0s 129us/step - loss: 0.1319 - acc: 0.9964 - val_loss: 0.1982 - val_acc: 0.9833
Epoch 68/300
278/278 [==========================

Epoch 121/300
278/278 [==============================] - 0s 118us/step - loss: 0.0588 - acc: 1.0000 - val_loss: 0.1350 - val_acc: 0.9667
Epoch 122/300
278/278 [==============================] - 0s 118us/step - loss: 0.0578 - acc: 1.0000 - val_loss: 0.1357 - val_acc: 0.9667
Epoch 123/300
278/278 [==============================] - 0s 115us/step - loss: 0.0572 - acc: 1.0000 - val_loss: 0.1339 - val_acc: 0.9667
Epoch 124/300
278/278 [==============================] - 0s 115us/step - loss: 0.0564 - acc: 1.0000 - val_loss: 0.1343 - val_acc: 0.9667
Epoch 125/300
278/278 [==============================] - 0s 118us/step - loss: 0.0557 - acc: 1.0000 - val_loss: 0.1329 - val_acc: 0.9667
Epoch 126/300
278/278 [==============================] - 0s 118us/step - loss: 0.0553 - acc: 1.0000 - val_loss: 0.1325 - val_acc: 0.9667
Epoch 127/300
278/278 [==============================] - 0s 111us/step - loss: 0.0545 - acc: 1.0000 - val_loss: 0.1332 - val_acc: 0.9667
Epoch 128/300
278/278 [==================

Epoch 181/300
278/278 [==============================] - 0s 161us/step - loss: 0.0340 - acc: 1.0000 - val_loss: 0.1119 - val_acc: 0.9750
Epoch 182/300
278/278 [==============================] - 0s 149us/step - loss: 0.0338 - acc: 1.0000 - val_loss: 0.1093 - val_acc: 0.9667
Epoch 183/300
278/278 [==============================] - 0s 140us/step - loss: 0.0334 - acc: 1.0000 - val_loss: 0.1115 - val_acc: 0.9583
Epoch 184/300
278/278 [==============================] - 0s 122us/step - loss: 0.0334 - acc: 1.0000 - val_loss: 0.1105 - val_acc: 0.9750
Epoch 185/300
278/278 [==============================] - 0s 118us/step - loss: 0.0329 - acc: 1.0000 - val_loss: 0.1110 - val_acc: 0.9750
Epoch 186/300
278/278 [==============================] - 0s 118us/step - loss: 0.0329 - acc: 1.0000 - val_loss: 0.1115 - val_acc: 0.9667
Epoch 187/300
278/278 [==============================] - 0s 122us/step - loss: 0.0326 - acc: 1.0000 - val_loss: 0.1080 - val_acc: 0.9833
Epoch 188/300
278/278 [==================

Epoch 241/300
278/278 [==============================] - 0s 161us/step - loss: 0.0229 - acc: 1.0000 - val_loss: 0.1013 - val_acc: 0.9833
Epoch 242/300
278/278 [==============================] - 0s 161us/step - loss: 0.0230 - acc: 1.0000 - val_loss: 0.1026 - val_acc: 0.9833
Epoch 243/300
278/278 [==============================] - 0s 161us/step - loss: 0.0228 - acc: 1.0000 - val_loss: 0.1034 - val_acc: 0.9833
Epoch 244/300
278/278 [==============================] - 0s 183us/step - loss: 0.0227 - acc: 1.0000 - val_loss: 0.0992 - val_acc: 0.9833
Epoch 245/300
278/278 [==============================] - 0s 176us/step - loss: 0.0227 - acc: 1.0000 - val_loss: 0.1015 - val_acc: 0.9833
Epoch 246/300
278/278 [==============================] - 0s 169us/step - loss: 0.0224 - acc: 1.0000 - val_loss: 0.1003 - val_acc: 0.9833


In [12]:
model.evaluate([test_X, te_pr], test_y)

171/171 [==============================] - 0s 99us/step


[0.15774249478804264, 0.9532163739204407]

In [13]:
model